# Setup

In [ ]:
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import itertools

_ = load_dotenv()

In [ ]:
from aocd import submit
from aocd.models import Puzzle

In [ ]:
puzzle = Puzzle(year=2022, day=12)

In [ ]:
example_input, example_soln_a, example_soln_b = (
    puzzle.examples[0].input_data,
    *puzzle.examples[0].answers,
)
_input = puzzle.input_data

In [ ]:
class MapGraph:
    def __init__(self, vertices, edges, inverted=False):
        self.vertices = vertices
        self.edges = edges

        if inverted:
            self.edges = {}

    def shortest_distance(self, start, end):
        visited = [start]

        layers = {0: [start]}
        layer = 0
        while end not in visited:
            layers[layer + 1] = []

            for vertex in layers[layer]:
                for neighbour in [edge[1] for edge in self.edges[vertex]]:
                    if neighbour not in visited:
                        layers[layer + 1].append(neighbour)
                        visited.append(neighbour)

            layer += 1

        return layer

    def shortest_distance_multiple(self, start, ends):
        visited = [start]

        layers = {0: [start]}
        layer = 0
        while True:
            layers[layer + 1] = []

            for vertex in layers[layer]:
                for neighbour in [edge[1] for edge in self.edges[vertex]]:
                    if neighbour not in visited:
                        layers[layer + 1].append(neighbour)
                        visited.append(neighbour)
                        if neighbour in ends:
                            return layer + 1
            layer += 1

In [ ]:
def data_to_df(input: str):
    letter_to_number = {
        letter: i
        for i, letter in list(
            enumerate(["S", *sorted("qwertyuiopasdfghjklzxcvbnm"), "E"])
        )
    }

    data = [[letter_to_number[y] for y in x] for x in input.split("\n")]

    map_df = pd.DataFrame(data)  # (20, 0) and (20, 107) are where we start and end
    return map_df


def start_end(df, num_letters=26):
    for coordinate in list(itertools.product(range(df.shape[0]), range(df.shape[1]))):
        if df.iloc[coordinate] == 0:
            start = coordinate
        elif df.iloc[coordinate] == 27:
            end = coordinate

    return start, end


def children(coordinate, df):
    df_width = df.shape[0]
    df_height = df.shape[1]
    children = [
        (coordinate, kid)
        for kid in [
            (coordinate[0] + 1, coordinate[1]),
            (coordinate[0] - 1, coordinate[1]),
            (coordinate[0], coordinate[1] + 1),
            (coordinate[0], coordinate[1] - 1),
        ]
        if kid[0] in range(df_width)
        and kid[1] in range(df_height)
        and df.iloc[kid] <= df.iloc[coordinate] + 1
    ]
    return children


def parents(coordinate, df):
    df_width = df.shape[0]
    df_height = df.shape[1]
    parents = [
        (coordinate, parent)
        for parent in [
            (coordinate[0] + 1, coordinate[1]),
            (coordinate[0] - 1, coordinate[1]),
            (coordinate[0], coordinate[1] + 1),
            (coordinate[0], coordinate[1] - 1),
        ]
        if parent[0] in range(df_width)
        and parent[1] in range(df_height)
        and df.iloc[coordinate] <= df.iloc[parent] + 1
    ]
    return parents


def coordinates(df):
    return list(itertools.product(range(df.shape[0]), range(df.shape[1])))


def connections(df, inverted=False):
    if inverted is False:
        return {coordinate: children(coordinate, df) for coordinate in coordinates(df)}
    else:
        return {coordinate: parents(coordinate, df) for coordinate in coordinates(df)}

# Part A

In [ ]:
def solution_a(input: str):
    map_df = data_to_df(input=input)
    graph = MapGraph(coordinates(map_df), connections(map_df))
    return graph.shortest_distance(start_end(map_df)[0], start_end(map_df)[1])

In [ ]:
print("Part A example solution:", solution_a(input=example_input))
print("Part A example answer:", example_soln_a)

In [ ]:
solution_a_output = solution_a(input=_input)
print("Part A solution:", solution_a_output, "\n" + "-" * 60)
submit(solution_a_output, day=12, year=2022, part="a")

# Part B

In [ ]:
def solution_b(input: str):
    map_df = data_to_df(input=input)
    graph = MapGraph(coordinates(map_df), connections(map_df))

    check = [y for y in graph.vertices if map_df.iloc[y] == 1]

    graph_inv = MapGraph(coordinates(map_df), connections(map_df, inverted=True))
    return graph_inv.shortest_distance_multiple(start_end(map_df)[1], check)

In [ ]:
print("Part B example solution:", solution_b(example_input))
print("Part B example answer:", example_soln_b)

In [ ]:
solution_b_output = solution_b(_input)
print("Part B solution:", solution_b_output, "\n" + "-" * 60)
submit(solution_b_output, day=12, year=2022, part="b")